# Phase 3: Feature Engineering
**Project:** National Rent Intelligence Engine
**Goal:** Transform raw statistics into "Investor Intelligence" (Yields, Trends, and Pressure Signals).

**Key Features Created:**
1.  **Cap Rate (Yield):** The gross ROI of a property (`Annual Rent / Price`).
2.  **Lags (Momentum):** What was the rent 1 year ago? 2 years ago?
3.  **Student Pressure:** The gap between Student Visa growth and Housing Supply growth.
4.  **Target Variable:** We shift rent *backwards* by 1 year to teach the AI to predict the future.

In [1]:
import pandas as pd
import numpy as np
import os

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# 1. AUTO-DETECT PATHS (Same logic as Phase 2)
if os.path.exists("data/processed"):
    PROCESSED_PATH = "data/processed"
elif os.path.exists("../data/processed"):
    PROCESSED_PATH = "../data/processed"
else:
    print("⚠️ Warning: Could not auto-detect paths. Defaulting to '../data/processed'")
    PROCESSED_PATH = "../data/processed"

OUTPUT_FILE = "national_training_data.csv"
FORECAST_FILE = "national_forecast_input.csv"

# Load the Master Dataset created in Phase 2
try:
    df = pd.read_csv(os.path.join(PROCESSED_PATH, "national_master_dataset.csv"))
    print(f"✅ Master Data Loaded: {len(df)} rows.")
    print(f"   Cities: {df['City'].unique()}")
except FileNotFoundError:
    print("❌ Error: 'national_master_dataset.csv' not found. Please run 01_data_merger.ipynb first.")

❌ Error: 'national_master_dataset.csv' not found. Please run 01_data_merger.ipynb first.
